In [1]:
import pandas as pd
df_train = pd.read_csv('train.csv', encoding='utf-8')
df_test = pd.read_csv('test.csv', encoding='utf-8')

In [2]:
import re
def preprocessor(text):
    text = re.sub('[\W]+', ' ', text.lower())
    return text

In [3]:
df_train['text'] = df_train['text'].apply(preprocessor)

In [4]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
df_train['author'] = class_le.fit_transform(df_train['author'].values)

In [5]:
from sklearn.model_selection import train_test_split
X, y = df_train.iloc[:, df_train.columns.get_loc('text')].values, \
       df_train.iloc[:, df_train.columns.get_loc('author')].values
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.3,
                                                   random_state=42,
                                                   stratify=y)

In [6]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords as sw
stopwords = sw.words('english')

[nltk_data] Downloading package stopwords to /home/gg/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /home/gg/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


#### Tokenize
Tokenize your sentences into separate words first.

In [7]:
def tokenizer(text):
    return text.split()

#### Stemming and Lemming helper functions
This functions will use stemming and lemmatization which reduces yours words into their root form.
Runner -> run
Running -> run

In [8]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [9]:
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()
def tokenizer_lemma(text):
    return [lemma.lemmatize(word) for word in text.split()]

#### Setup
We will use pipeline to better organize our code. Pipeline will allow us establish our workflow easily.
We will also use GridSearch or RandomSearch with k-fold to find optimized parameters for our model. We will use neg_log_loss instead of accuracy because that is what Kaggle is using to grade submissions.

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(strip_accents=None,
                       lowercase=False,
                       preprocessor=None)

param_grid = {'vect__ngram_range': [(1,1)],
              'vect__stop_words': [stopwords],
              'vect__tokenizer': [tokenizer,
                                  tokenizer_porter,
                                  tokenizer_lemma],
              'clf__penalty': ['l1', 'l2'],
              'clf__C': [0.001, 0.01]}

pipe = Pipeline([
                 ('vect', count),
                 ('clf', LogisticRegression(random_state=42))
                ])

model = GridSearchCV(pipe, param_grid, scoring='neg_log_loss', cv=10, verbose=1, n_jobs=-1)
model.fit(X_train, y_train)

Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  1.1min finished


GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        s...alty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__ngram_range': [(1, 1)], 'vect__tokenizer': [<function tokenizer at 0x7f7ebb5b77d0>, <function tokenizer_porter at 0x7f7e49c231b8>, <function tokenizer_lemma at 0x7f7e49c23578>], 'clf__penalty': ['l1', 'l2'], 'clf__C': [0.001, 0.01], 'vect__stop_words': [[u'i', u'me', u'my', u'myse... u"shouldn't", u'wasn', u"wasn't", u'weren', u"weren't", u'won', u"won't", u'wouldn', u"wouldn't"]]},
       pre_dispatch='2*n_jobs', refit=True, return_t

In [11]:
print(model.best_estimator_)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=[u'i', ...alty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])


In [12]:
print(model.best_params_)

{'vect__ngram_range': (1, 1), 'vect__tokenizer': <function tokenizer_porter at 0x7f7e49c231b8>, 'clf__penalty': 'l2', 'clf__C': 0.01, 'vect__stop_words': [u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u"you're", u"you've", u"you'll", u"you'd", u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u"she's", u'her', u'hers', u'herself', u'it', u"it's", u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u"that'll", u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on

In [13]:
print(model.best_score_)

-0.8464369815325335


#### Check for overfitting
When you notice that training data has a higher accuracy than the test data, that is one indicator of overfitting.
One method to solve this, is to play around with the C parameter of your logistic regression model. The lower the C parameter the more it will be penalized. The C parameter is an inverse of the regularization parameter.

You can use the confusion matrix to get true and false positives of your model.

In [14]:
y_pred_train = model.best_estimator_.predict(X_train)

In [15]:
from sklearn.metrics import confusion_matrix
conf_matrix_train = confusion_matrix(y_true=y_train, y_pred=y_pred_train)
print(conf_matrix_train)

[[5031  215  284]
 [1252 2445  247]
 [1380  205 2646]]


In [16]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_pred_train)

0.7385625684056913

In [17]:
y_pred_test = model.best_estimator_.predict(X_test)

In [18]:
conf_matrix_test = confusion_matrix(y_true=y_test, y_pred=y_pred_test)
print(conf_matrix_test)

[[2112  114  144]
 [ 603  955  133]
 [ 638  125 1050]]


In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_test)

0.7008852570650324

#### Save the model
Save the model with pickle, this way you can use this pickle object later for more fun! Ensemble learning!

WARNING: Pickle objects can contain viruses, I recommend building your own pickle objects.

In [24]:
import pickle
import os
pickle.dump(model.best_estimator_,
           open(os.path.join('./pkl_objects', 'bag_1gram_lr_pipe.pkl'), 'wb'),
           protocol=2)